In [19]:
from datetime import datetime
import os

import reflexio

client = reflexio.ReflexioClient()
client.api_key = client.login("peakmojo", "s").access_token


## Publish interaction

publish text interaction example

In [17]:
client.publish_interaction(
    request_id="your_request_id",  # for attribution for profile sources
    user_id="your_user_id",
    interaction_requests=[
        {
            'timestamp': 1738572096,
            'content': 'Hey this is Yi calling from kiku sushi, we offer salmon sushi, are you interested?',
            'role': 'Sales Rep'
        }
    ]
)

PublishUserInteractionResponse(success=True, message='')

### batch publish interactions

In [6]:
import pandas as pd
import re

def parse_dialogue(text):
    # Find the dialogue section
    dialogue_match = re.search(r'<DIALOG_EXAMPLE>(.*?)</DIALOG_EXAMPLE>', text, re.DOTALL)
    if not dialogue_match:
        return pd.DataFrame(columns=['Role', 'Content'])
    
    dialogue = dialogue_match.group(1).strip()
    
    # Split into conversation turns
    turns = dialogue.split('\n\n')
    
    # Process each turn
    conversation_data = []
    for turn in turns:
        if ':' in turn:
            # Split at first colon to separate role from content
            role, content = turn.split(':', 1)
            
            # Clean up the role and content
            role = role.strip()
            content = content.strip()
            
            # Map roles to consistent format
            role = 'Customer' if role == 'Customer' else 'Sales Rep'
            
            conversation_data.append({
                'Role': role,
                'Content': content
            })
    
    # Create DataFrame
    df = pd.DataFrame(conversation_data)
    return df

In [16]:
fp  = '../experiments/peakmojo/h2gjw2n6ud/ibrssunqfx_updated.txt'

def add_interaction_from_fp(fp):
    with open(fp, 'r') as f:
        raw_conversation = f.read()
        df = parse_dialogue(raw_conversation)
        interactions = list(df[df['Role'] == 'Sales Rep']['Content'])
        user_id = os.path.splitext(os.path.basename(fp))[0]
        print(f"processing user id: {user_id}")

    
        for i, interaction in enumerate(interactions):
            interaction_requests = [{
                'timestamp': int(datetime.utcnow().timestamp()),
                'content': interaction,
                'role': 'Sales Rep'
            }]
            
            client.publish_interaction(
            request_id=f"request_{user_id}_{i}",  # for attribution for profile sources
            user_id=user_id,
            interaction_requests=interaction_requests)
            print(interaction_requests)



In [18]:
add_interaction_from_fp(fp)

processing user id: ibrssunqfx_updated
[{'timestamp': 1738679975, 'text_interaction': "Hey, may I speak with Bob? Hello? Who's this?", 'role': 'Sales Rep'}]
[{'timestamp': 1738679976, 'text_interaction': "Hey, Bob, this is Kiara. I'm calling with Fundera. It looks like we helped you secure some financing with our partners at Reliant Funding back in May of 2020.", 'role': 'Sales Rep'}]
[{'timestamp': 1738679978, 'text_interaction': 'Yeah, I just wanted to reach out to you to see if you had any upcoming projects where some additional working capital could be helpful.', 'role': 'Sales Rep'}]
[{'timestamp': 1738679979, 'text_interaction': 'Yeah, for sure. You want to make some upgrades and extend your product line? We could definitely help with that. Do you happen to still have that loan with Reliant Funding?', 'role': 'Sales Rep'}]
[{'timestamp': 1738679981, 'text_interaction': 'We have a variety of options. We have short term loans, medium term loans, as well as the SBA 7A, which a lot o

## Delete operations

In [22]:
# delete interaction
client.delete_interaction(
    user_id="your_user_id",
    interaction_id="your_user_id_your_request_id_762bddb4"
)

DeleteUserInteractionResponse(success=True, message='')

In [21]:
# delete user profile
client.delete_profile(
    user_id="your_user_id",
    profile_id='dc4925f6-92b2-4907-9bff-d2c103df3050'
)

DeleteUserProfileResponse(success=True, message='')

## Get changelog, user profiles and interactions

In [ ]:
logs = client.get_profile_change_log()
logs.model_dump()

In [ ]:
interactions = client.get_interactions(
    request = {
        "user_id": "ibrssunqfx_updated",
        "top_k": 4
    }
)
interactions.model_dump()

In [ ]:
profiles = client.get_profiles(
    request = {
        "user_id": "ibrssunqfx_updated",
        "top_k": 4
    }
)
profiles.model_dump()

# analyze profiles and changelog

In [21]:
profile_df = pd.read_csv('./data/peakmojo/user_profiles.csv')
interaction_df = pd.read_csv('./data/peakmojo/interactions.csv')
change_logs_df = pd.read_csv('./data/peakmojo/change_logs.csv')

In [20]:
profile_df

,profile_id,user_id,profile_content,last_modified_timestamp,generated_from_request_id,profile_time_to_live,expiration_timestamp
0,757ea07a-00d7-4f7f-b918-837d0663e40c,ibrssunqfx_updated,Kiara from Fundera,1738679978,request_ibrssunqfx_updated_1,infinity,253402329600
1,d7d7deb4-1663-4c74-8b91-17ee23188e52,ibrssunqfx_updated,Helped secure financing with Reliant Funding in May 2020,1738679978,request_ibrssunqfx_updated_1,infinity,253402329600
2,46785e94-1257-4df3-b601-afb4298970b6,ibrssunqfx_updated,"Sales rep's intent is to discuss loan options including short term loans, medium term loans, SBA 7A, and lines of credit",1738679984,request_ibrssunqfx_updated_4,infinity,253402329600
3,d0dc4c7e-c7d2-46f9-b109-4a5196ea343b,ibrssunqfx_updated,Sales rep's intent is to provide working capital for expansion goals,1738679987,request_ibrssunqfx_updated_5,infinity,253402329600
4,d16529a5-cf82-47f9-8075-212c7b8b9e8f,ibrssunqfx_updated,Lending decision can be made within 24 to 48 hours,1738679987,request_ibrssunqfx_updated_5,one_week,1739284787
5,7ed27cb4-754e-471b-973d-fdd3a862e6a7,ibrssunqfx_updated,Process requires four months of the most recent bank statements and some credit information,1738679987,request_ibrssunqfx_updated_5,one_month,1741271987
6,828b6db8-dd33-4bbd-8bae-475f0a71b9e3,ibrssunqfx_updated,"First funded amount was around $76,000",1738679990,request_ibrssunqfx_updated_6,infinity,253402329600
7,8e671eaf-7daa-47d4-8b60-22cd2d00e851,ibrssunqfx_updated,Loan options available up to a couple hundred thousand dollars,1738679990,request_ibrssunqfx_updated_6,infinity,253402329600
8,8aac7663-3fe9-416a-b2ea-bf40c1dd325f,ibrssunqfx_updated,Sales rep's intent is to evaluate most recent bank statements and credit for funding qualification,1738679993,request_ibrssunqfx_updated_7,one_month,1741271993
9,14107dd7-3bde-4618-bee8-4d674c2b5b77,ibrssunqfx_updated,Business credit and industry are factors in funding qualification,1738679993,request_ibrssunqfx_updated_7,one_month,1741271993


In [22]:
interaction_df

,interaction_id,user_id,request_id,timestamp,role,text_interaction,user_action,user_action_description,interacted_image_url,image_encoding
0,ibrssunqfx_updated_request_ibrssunqfx_updated_0_9c3ac3ea,ibrssunqfx_updated,request_ibrssunqfx_updated_0,1738679975,Sales Rep,"Hey, may I speak with Bob? Hello? Who's this?",none,NaN,NaN,NaN
1,ibrssunqfx_updated_request_ibrssunqfx_updated_1_eb7047ec,ibrssunqfx_updated,request_ibrssunqfx_updated_1,1738679976,Sales Rep,"Hey, Bob, this is Kiara. I'm calling with Fundera. It looks like we helped you secure some financing with our partners at Reliant Funding back in May of 2020.",none,NaN,NaN,NaN
2,ibrssunqfx_updated_request_ibrssunqfx_updated_2_42b093f0,ibrssunqfx_updated,request_ibrssunqfx_updated_2,1738679978,Sales Rep,"Yeah, I just wanted to reach out to you to see if you had any upcoming projects where some additional working capital could be helpful.",none,NaN,NaN,NaN
3,ibrssunqfx_updated_request_ibrssunqfx_updated_3_f2d91b02,ibrssunqfx_updated,request_ibrssunqfx_updated_3,1738679979,Sales Rep,"Yeah, for sure. You want to make some upgrades and extend your product line? We could definitely help with that. Do you happen to still have that loan with Reliant Funding?",none,NaN,NaN,NaN
4,ibrssunqfx_updated_request_ibrssunqfx_updated_4_aeae45e1,ibrssunqfx_updated,request_ibrssunqfx_updated_4,1738679981,Sales Rep,"We have a variety of options. We have short term loans, medium term loans, as well as the SBA 7A, which a lot of entrepreneurs with profiles similar to yours are interested in. And we also have lines of credit. Do either of those sound like they might be interesting to you?",none,NaN,NaN,NaN
5,ibrssunqfx_updated_request_ibrssunqfx_updated_5_d4cc08ab,ibrssunqfx_updated,request_ibrssunqfx_updated_5,1738679984,Sales Rep,"These loans would be able to help by providing you with the working capital that you need for your expansion goals. The terms are dependent on your credit, as well as your business profile. The process works by connecting you with your funding advisor who will let you know the things they need from you. Essentially it's just four months of the most recent bank statements and some credit information and then we can get you a lending decision within 24 to 48 hours. Is that a timeline that works for you?",none,NaN,NaN,NaN
6,ibrssunqfx_updated_request_ibrssunqfx_updated_6_b8777555,ibrssunqfx_updated,request_ibrssunqfx_updated_6,1738679987,Sales Rep,"It depends on which loan you qualify for, but looks like your first funded amount was around $76,000. We definitely have that as an opportunity as well as up to a couple hundred thousand dollars. It depends on what you need and what you qualify for. About how much do you need?",none,NaN,NaN,NaN
7,ibrssunqfx_updated_request_ibrssunqfx_updated_7_79ec576a,ibrssunqfx_updated,request_ibrssunqfx_updated_7,1738679990,Sales Rep,"We will take a look at your most recent bank statements and your credit. Business credit affects it as well as the industry that you're in. Those are a couple of factors. But again, your funding advisor will provide more information on what you qualify for.",none,NaN,NaN,NaN
8,ibrssunqfx_updated_request_ibrssunqfx_updated_8_ac649b59,ibrssunqfx_updated,request_ibrssunqfx_updated_8,1738679993,Sales Rep,I'm not certain for sure. That's something that your funding advisor would be able to provide you with further information on. What is the time frame that you're looking to receive funding in?,none,NaN,NaN,NaN
9,ibrssunqfx_updated_request_ibrssunqfx_updated_9_8a627bc5,ibrssunqfx_updated,request_ibrssunqfx_updated_9,1738679993,Sales Rep,"Perfect. What I'll do next is pass this information along and connect you with your funding advisor. Her name is Marie, and she'll let you know what all items are needed to get some offers on the table. Does that sound good?",none,NaN,NaN,NaN


In [32]:
import json
change_logs_df['added_profile_texts'] = change_logs_df.apply(lambda x : [profile['profile_content'] for profile in json.loads(x["added_profiles"])], axis=1)
change_logs_df['removed_profile_texts'] = change_logs_df.apply(lambda x : [profile['profile_content'] for profile in json.loads(x["removed_profiles"])], axis=1)

In [34]:
change_logs_df.sort_values(by='request_id', inplace=True)
change_logs_df


,id,user_id,request_id,created_at,added_profiles,removed_profiles,mentioned_profiles,added_profile_texts,removed_profile_texts
6,1,ibrssunqfx_updated,request_ibrssunqfx_updated_1,2025-02-04 06:39:38,"[{""profile_id"": ""757ea07a-00d7-4f7f-b918-837d0663e40c"", ""user_id"": ""ibrssunqfx_updated"", ""profile_content"": ""Kiara from Fundera"", ""last_modified_timestamp"": 1738679978, ""generated_from_request_id"": ""request_ibrssunqfx_updated_1"", ""profile_time_to_live"": ""infinity"", ""expiration_timestamp"": 253402329600}, {""profile_id"": ""d7d7deb4-1663-4c74-8b91-17ee23188e52"", ""user_id"": ""ibrssunqfx_updated"", ""profile_content"": ""Helped secure financing with Reliant Funding in May 2020"", ""last_modified_timestamp"": 1738679978, ""generated_from_request_id"": ""request_ibrssunqfx_updated_1"", ""profile_time_to_live"": ""infinity"", ""expiration_timestamp"": 253402329600}]",[],[],"[Kiara from Fundera, Helped secure financing with Reliant Funding in May 2020]",[]
5,2,ibrssunqfx_updated,request_ibrssunqfx_updated_3,2025-02-04 06:39:41,"[{""profile_id"": ""03041295-a224-42d7-a6cc-1463485bf4dd"", ""user_id"": ""ibrssunqfx_updated"", ""profile_content"": ""Sales rep's intent is to help with upgrades and extending the product line"", ""last_modified_timestamp"": 1738679981, ""generated_from_request_id"": ""request_ibrssunqfx_updated_3"", ""profile_time_to_live"": ""infinity"", ""expiration_timestamp"": 253402329600}]",[],[],[Sales rep's intent is to help with upgrades and extending the product line],[]
4,3,ibrssunqfx_updated,request_ibrssunqfx_updated_4,2025-02-04 06:39:44,"[{""profile_id"": ""46785e94-1257-4df3-b601-afb4298970b6"", ""user_id"": ""ibrssunqfx_updated"", ""profile_content"": ""Sales rep's intent is to discuss loan options including short term loans, medium term loans, SBA 7A, and lines of credit"", ""last_modified_timestamp"": 1738679984, ""generated_from_request_id"": ""request_ibrssunqfx_updated_4"", ""profile_time_to_live"": ""infinity"", ""expiration_timestamp"": 253402329600}]","[{""profile_id"": ""03041295-a224-42d7-a6cc-1463485bf4dd"", ""user_id"": ""ibrssunqfx_updated"", ""profile_content"": ""Sales rep's intent is to help with upgrades and extending the product line"", ""last_modified_timestamp"": 1738679981, ""generated_from_request_id"": ""request_ibrssunqfx_updated_3"", ""profile_time_to_live"": ""infinity"", ""expiration_timestamp"": 253402329600}]",[],"[Sales rep's intent is to discuss loan options including short term loans, medium term loans, SBA 7A, and lines of credit]",[Sales rep's intent is to help with upgrades and extending the product line]
3,4,ibrssunqfx_updated,request_ibrssunqfx_updated_5,2025-02-04 06:39:47,"[{""profile_id"": ""d0dc4c7e-c7d2-46f9-b109-4a5196ea343b"", ""user_id"": ""ibrssunqfx_updated"", ""profile_content"": ""Sales rep's intent is to provide working capital for expansion goals"", ""last_modified_timestamp"": 1738679987, ""generated_from_request_id"": ""request_ibrssunqfx_updated_5"", ""profile_time_to_live"": ""infinity"", ""expiration_timestamp"": 253402329600}, {""profile_id"": ""d16529a5-cf82-47f9-8075-212c7b8b9e8f"", ""user_id"": ""ibrssunqfx_updated"", ""profile_content"": ""Lending decision can be made within 24 to 48 hours"", ""last_modified_timestamp"": 1738679987, ""generated_from_request_id"": ""request_ibrssunqfx_updated_5"", ""profile_time_to_live"": ""one_week"", ""expiration_timestamp"": 1739284787}, {""profile_id"": ""7ed27cb4-754e-471b-973d-fdd3a862e6a7"", ""user_id"": ""ibrssunqfx_updated"", ""profile_content"": ""Process requires four months of the most recent bank statements and some credit information"", ""last_modified_timestamp"": 1738679987, ""generated_from_request_id"": ""request_ibrssunqfx_updated_5"", ""profile_time_to_live"": ""one_month"", ""expiration_timestamp"": 1741271987}]",[],[],"[Sales rep's intent is to provide working capital for expansion goals, Lending decision can be made within 24 to 48 hours, Process requires four months of

In [45]:
def get_profiles_so_far(row):
    profiles_so_far = []
    idx = int(row['request_id'].split('_')[-1])
    for _, change_log in change_logs_df.iterrows():
        change_log_idx = int(change_log['request_id'].split('_')[-1])
        if idx >= change_log_idx:
            for remove_text in change_log['removed_profile_texts']:
                if remove_text in profiles_so_far:
                    profiles_so_far.remove(remove_text)
                else:
                    print(f"missing {remove_text}")

            
            profiles_so_far.extend(change_log['added_profile_texts'])
        else:
            break

    return profiles_so_far
        
interaction_df['profile_so_far'] = interaction_df.apply(lambda x: get_profiles_so_far(x), axis=1)

In [46]:
interaction_df[["request_id", "content", "profile_so_far"]]

,request_id,text_interaction,profile_so_far
0,request_ibrssunqfx_updated_0,"Hey, may I speak with Bob? Hello? Who's this?",[]
1,request_ibrssunqfx_updated_1,"Hey, Bob, this is Kiara. I'm calling with Fundera. It looks like we helped you secure some financing with our partners at Reliant Funding back in May of 2020.","[Kiara from Fundera, Helped secure financing with Reliant Funding in May 2020]"
2,request_ibrssunqfx_updated_2,"Yeah, I just wanted to reach out to you to see if you had any upcoming projects where some additional working capital could be helpful.","[Kiara from Fundera, Helped secure financing with Reliant Funding in May 2020]"
3,request_ibrssunqfx_updated_3,"Yeah, for sure. You want to make some upgrades and extend your product line? We could definitely help with that. Do you happen to still have that loan with Reliant Funding?","[Kiara from Fundera, Helped secure financing with Reliant Funding in May 2020, Sales rep's intent is to help with upgrades and extending the product line]"
4,request_ibrssunqfx_updated_4,"We have a variety of options. We have short term loans, medium term loans, as well as the SBA 7A, which a lot of entrepreneurs with profiles similar to yours are interested in. And we also have lines of credit. Do either of those sound like they might be interesting to you?","[Kiara from Fundera, Helped secure financing with Reliant Funding in May 2020, Sales rep's intent is to discuss loan options including short term loans, medium term loans, SBA 7A, and lines of credit]"
5,request_ibrssunqfx_updated_5,"These loans would be able to help by providing you with the working capital that you need for your expansion goals. The terms are dependent on your credit, as well as your business profile. The process works by connecting you with your funding advisor who will let you know the things they need from you. Essentially it's just four months of the most recent bank statements and some credit information and then we can get you a lending decision within 24 to 48 hours. Is that a timeline that works for you?","[Kiara from Fundera, Helped secure financing with Reliant Funding in May 2020, Sales rep's intent is to discuss loan options including short term loans, medium term loans, SBA 7A, and lines of credit, Sales rep's intent is to provide working capital for expansion goals, Lending decision can be made within 24 to 48 hours, Process requires four months of the most recent bank statements and some credit information]"
6,request_ibrssunqfx_updated_6,"It depends on which loan you qualify for, but looks like your first funded amount was around $76,000. We definitely have that as an opportunity as well as up to a couple hundred thousand dollars. It depends on what you need and what you qualify for. About how much do you need?","[Kiara from Fundera, Helped secure financing with Reliant Funding in May 2020, Sales rep's intent is to discuss loan options including short term loans, medium term loans, SBA 7A, and lines of credit, Sales rep's intent is to provide working capital for expansion goals, Lending decision can be made within 24 to 48 hours, Process requires four months of the most recent bank statements and some credit information, First funded amount was around $76,000, Loan options available up to a couple hundred thousand dollars]"
7,request_ibrssunqfx_updated_7,"We will take a look at your most recent bank statements and your credit. Business credit affects it as well as the industry that you're in. Those are a couple of factors. But again, your funding advisor will provide more information on what you qualify for.","[Kiara from Fundera, Helped secure financing with Reliant Funding in May 2020, Sales rep's intent is to discuss loan options including short term loans, medium term loans, SBA 7A, and lines of credit, Sales rep's intent is to provide working capital for expansion goals, Lending decision can be made within 24 to 48 hours, Process requires four months of the most recent bank statements and som

In [59]:
import tiktoken
enc = tiktoken.encoding_for_model("gpt-4o")

In [60]:
len(enc.encode("Kiara from Fundera, Helped secure financing with Reliant Funding in May 2020, Sales rep's intent is to discuss loan options including short term loans, medium term loans, SBA 7A, and lines of credit, Sales rep's intent is to provide working capital for expansion goals, Lending decision can be made within 24 to 48 hours, Process requires four months of the most recent bank statements and some credit information, First funded amount was around $76,000, Loan options available up to a couple hundred thousand dollars, Sales rep's intent is to evaluate most recent bank statements and credit for funding qualification, Business credit and industry are factors in funding qualification, Funding advisor will provide more information on qualification, Marie is the funding advisor"))

151